# 03 - Merge With Hashing Info

In [1]:
from libraries import *
from parameters import *

from pathlib import Path
from tqdm.auto import tqdm

In [2]:
%load_ext rpy2.ipython

In [3]:
os.getcwd()
os.chdir(projectDir)

***

In [4]:
adata = sc.read(par_save_filename_1)

In [5]:
hash_ads = {}
feats = {}

for d in tqdm(sorted([x for x in Path('data').iterdir() if x.is_dir() and x.name.startswith('E')])):
    sample = d.name.replace('_', '-')
    roundlow = sample.split('-')[0].lower()
    chrom = sample.split('-')[1].lower()
    
    hash_path = Path('data') / sample.replace('-', '_') / f'{roundlow}_ch{chrom}_mixhash' / f'{roundlow}_ch{chrom}_mixhash_demux.h5ad'
    if hash_path.is_file():
        print(sample)
        hash_ads[sample] = sc.read(hash_path)
        
    feat_path = Path(f'data/{sample.replace("-", "_")}/R{roundlow[1:]}_Ch{chrom}_FeatBarc/R{roundlow[1:]}_Ch{chrom}_FeatBarc.csv')
    if feat_path.is_file():
        print(sample)
        feats[sample] = pd.read_csv(feat_path)

  0%|          | 0/46 [00:00<?, ?it/s]

E3-1
E3-1
E3-10
E3-10
E3-11
E3-11
E3-12
E3-12
E3-13
E3-13
E3-14
E3-14
E3-15
E3-15
E3-16
E3-16
E3-17
E3-17
E3-18
E3-18
E3-19
E3-19
E3-2
E3-2
E3-20
E3-20
E3-21
E3-21
E3-22
E3-22
E3-23
E3-23
E3-24
E3-24
E3-3
E3-3
E3-4
E3-4
E3-5
E3-5
E3-6
E3-6
E3-7
E3-7
E3-8
E3-8
E3-9
E3-9
E4-1
E4-1
E4-10
E4-10
E4-11
E4-11
E4-12
E4-12
E4-13
E4-13
E4-14
E4-14
E4-15
E4-15
E4-16
E4-16
E4-17
E4-17
E4-19
E4-19
E4-2
E4-2
E4-20
E4-20
E4-21
E4-21
E4-22
E4-22
E4-24
E4-24
E4-3
E4-3
E4-4
E4-4
E4-5
E4-5
E4-6
E4-6
E4-7
E4-7
E4-8
E4-8
E4-9
E4-9


## Merge hash IDs

In [6]:
%%time

hash_ad = sc.AnnData.concatenate(*hash_ads.values(), 
                                 batch_key='mixhash', 
                                 batch_categories=list(hash_ads.keys()))
hash_ad

/home/eraslab1/miniconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


CPU times: user 17.4 s, sys: 13.4 s, total: 30.8 s
Wall time: 30.7 s


AnnData object with n_obs × n_vars = 1258502 × 31053
    obs: 'Channel', 'n_genes', 'n_counts', 'demux_type', 'assignment', 'mixhash'
    var: 'gene_ids', 'robust'
    obsm: 'raw_probs'

In [7]:
hash_ad.obs.index = ['-'.join([x.split('-')[0], 
                               '1', 
                               x.split('-')[1], 
                               x.split('-')[2]]) for x in hash_ad.obs.index]

In [8]:
adata.obs['demux_type'] = hash_ad.obs.demux_type
adata.obs['hash_assignment'] = hash_ad.obs.assignment

In [9]:
adata.obs['hash_assignment'] = adata.obs['hash_assignment'].astype('category')

Filter out doublets and multiplets


In [10]:
adata = adata[adata.obs.demux_type == 'singlet']

In [11]:
adata.write(par_save_filename_1)